In [115]:
from scipy.io import loadmat, savemat
import torch
import torch.nn.functional as F
import numpy as np
import time
import yaml

import os
import matplotlib.pyplot as plt
from sdf.robot_sdf import RobotSdfCollisionNet

In [116]:
device = torch.device('cpu', 0)
tensor_args = {'device': device, 'dtype': torch.float32}
data = loadmat('../data-sampling/datasets/data_mesh.mat')['dataset']
print(data.shape)

(4950000, 19)


In [117]:
L1 = 0
L2 = int(1.0 * data.shape[0])
n_size = L2
train_ratio = 0.98
test_ratio = 0.01
val_ratio = 1 - train_ratio - test_ratio
idx_train = np.arange(0, int(n_size * train_ratio))
idx_val = np.arange(idx_train[-1] + 1, int(n_size * (train_ratio + test_ratio)))
idx_test = np.arange(idx_val[-1] + 1, int(n_size))

print("Number of train data      : ", idx_train.shape[0])
print("Number of validation data : ", idx_val.shape[0])
print("Number of test data       : ", idx_test.shape[0])

Number of train data      :  4851000
Number of validation data :  49500
Number of test data       :  49500


In [118]:
x = torch.Tensor(data[L1:L2, 0:10]).to(device, dtype=torch.float16)
y = 100 * torch.Tensor(data[L1:L2, 10:]).to(device, dtype=torch.float16)
dof = x.shape[1]
s = 256
n_layers = 5
batch_size = 50000
p = 0.1
skips = []
# fname = 'sdf_%dx%d_mesh_%d_drop_%1f.pt'%(s,n_layers,batch_size, p)
fname = 'model/sdf_%dx%d_mesh_%d.pt'%(s,n_layers,batch_size)
if skips == []:
    n_layers-=1
nn_model = RobotSdfCollisionNet(in_channels=dof, out_channels=y.shape[1], layers=[s] * n_layers, skips=skips, dropout_ratio=p)
nn_model.load_weights(fname, tensor_args)
nn_model.model.to(**tensor_args)
model = nn_model.model

Weights loaded!


In [119]:
x_test = x[idx_test, :]
y_test = y[idx_test, :]
y_test_labels = y[idx_test, :]
y_test_labels[y_test_labels<=3] = 0
y_test_labels[y_test_labels>3] = 1
y_test_labels = y_test_labels.cpu().detach().numpy()
print(y_test_labels)

[[0. 0. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 1. 0.]
 [1. 1. 1. ... 1. 1. 0.]]


In [120]:
y_pred = nn_model.compute_signed_distance(x_test)
test_loss = F.mse_loss(y_pred, y_test, reduction='mean')
print("Test Loss : %f [cm^2]" %(test_loss))
y_pred = y_pred.cpu().detach().numpy()

Test Loss : 0.759513 [cm^2]


In [121]:
y_pred = nn_model.compute_signed_distance(x_test)
y_pred = y_pred.cpu().detach().numpy()
y_pred_labels = np.copy(y_pred)
y_pred_labels[y_pred_labels<=3] = 0
y_pred_labels[y_pred_labels>3] = 1
print(y_pred_labels)

[[0. 1. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 0.]]


In [122]:
from sklearn.metrics import confusion_matrix
conf_matrix = np.zeros((2,2,9))
for i in range(0,9):
    conf_matrix[:,:,i] = confusion_matrix(y_test_labels[:,i], y_pred_labels[:,i])
print(conf_matrix[:,:,8])
# [real=0 and pred=0] : TN, [real=0 and pred=1] : FP
# [real=1 and pred=0] : FN, [real=1 and pred=1] : TP

[[ 4972.   864.]
 [  521. 43143.]]


In [123]:
for i in range(0,9):
    print("Link %d : " %(i+1))
    print("\tTPR = %f" %(conf_matrix[1,1,i]/(conf_matrix[1,0,i]+conf_matrix[1,1,i])))
    print("\tTNR = %f" %(conf_matrix[0,0,i]/(conf_matrix[0,1,i]+conf_matrix[0,0,i])))

Link 1 : 
	TPR = 0.999160
	TNR = 0.958256
Link 2 : 
	TPR = 0.998411
	TNR = 0.957181
Link 3 : 
	TPR = 0.998043
	TNR = 0.951002
Link 4 : 
	TPR = 0.996239
	TNR = 0.951109
Link 5 : 
	TPR = 0.995179
	TNR = 0.955621
Link 6 : 
	TPR = 0.991249
	TNR = 0.881495
Link 7 : 
	TPR = 0.988475
	TNR = 0.884758
Link 8 : 
	TPR = 0.992236
	TNR = 0.868403
Link 9 : 
	TPR = 0.988068
	TNR = 0.851953
